### Change the 'Filename' and 'Folder' of the file you want to convert and run the code

`!jupyter nbconvert` does most of the converting work but there are leftover formatting issues.

This code removes also the magic commands (strating with %) and makes the script readable by removing #ln(Cell number), markdown code and comments.

This also wraps the code under a main() call.

Isses can be if a cell starts with a magic comman as with %time.

In [56]:
Folder = ''+''  ## '+' if the same directory
Filename = 'CoffeaJERC-Andris'  #without .ipynb

In [57]:
PathFile = Folder + Filename

In [58]:
!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['parameters']" \
    --TemplateExporter.exclude_markdown=True \
    --to python {PathFile+".ipynb"}

/cvmfs/sft-nightlies.cern.ch/lcg/views/dev4/Mon/x86_64-centos7-gcc11-opt/lib/python3.9/site-packages/traitlets/traitlets.py:2556: FutureWarning: --TagRemovePreprocessor.remove_cell_tags=['parameters'] for containers is deprecated in traitlets 5.0. You can pass `--TagRemovePreprocessor.remove_cell_tags item` ... multiple times to add items to a list.
  warn(
[NbConvertApp] Converting notebook CoffeaJERC-Andris.ipynb to python
[NbConvertApp] Writing 33457 bytes to CoffeaJERC-Andris.py


In [59]:
import sys
sys.executable

'/cvmfs/sft-nightlies.cern.ch/lcg/latest/Python/3.9.12-9a1bc/x86_64-centos7-gcc11-opt/bin/python3'

In [60]:
import re
from pathlib import Path
import numpy as np

In [61]:
def flatten(foo):
    for x in foo:
        if hasattr(x, '__iter__') and not isinstance(x, str):
            for y in flatten(x):
                yield y
        else:
            yield x

In [62]:
def LineCheck(line):
    if (len(line) == 0 or (line[0] != '#') and (not('InteractiveShell' in line)) and (not('Image' in line))):
#         print("Here1")
        if 'get_ipython()' in line:
#             print("Here2")
            #             if 'get_ipython().run_cell_magic(\'time\'' in line:
            if line[:42] == "get_ipython().run_cell_magic('time', '', '":
#                 print("Here3")
                line_split = line[42:-2].split('\\n')
#                 [li.replace("\\'", "'") for li in line_split]
                return line_split
        else:
            return line

In [63]:
filename = Path.cwd() / (PathFile + '.py')
code_text = filename.read_text().split('\n')
lines = [LineCheck(line) for line in code_text]
lines = [x for x in lines if x is not None]  ###Remove all None created by previous line
lines = [i for i in flatten(lines)]

lines2 = []
was_empty = 0
for li in lines:
    if len(li)==0 and not was_empty:
        lines2.append('    '+li)
        was_empty=1
    elif len(li)!=0:
        lines2.append('    '+li)
        was_empty=0

lines = lines2
        # lines = ['    '+i for i in lines if i!='\n']
initial_comment_lines = ["### "+PathFile+".py",
                         "### File automatically converted using ConvertJupyterToPy.ipynb from "+PathFile+".ipynb",
                         "### No comments or formatting is preserved by the transfer."]
lines = initial_comment_lines+["def main():"]+lines

main_lines = ['if __name__ == "__main__":',
              '    main()' ]
lines = lines+main_lines

clean_code = '\n'.join(lines)
# clean_code = re.sub(r'\n{2,}', '\n\n', clean_code)
filename.write_text(clean_code.strip())

29632